In [9]:
import csv
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim import corpora
from gensim.models import Word2Vec as w2v
import codecs

labels = []
textsPOS = []
textsNEG = []
texts = []
with codecs.open("spam.csv", "r", "ISO-8859-1") as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        try:
                if row[0] == 'ham':
                        labels.append(0)
                        textsNEG.append(row[1])
                elif row[0] == 'spam':
                        labels.append(1)
                        textsPOS.append(row[1])
                texts.append(row[1])
        except:
            pass

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in texts]   
dictionary = corpora.Dictionary(doc_clean)

model = w2v(doc_clean, size=30, window=5, min_count=5)
W = model.wv.vocab.items()

doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

Lda = gensim.models.ldamodel.LdaModel

ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

print(ldamodel.print_topics(num_topics=10, num_words=4))

[(0, '0.030*"call" + 0.010*"2" + 0.009*"free" + 0.009*"txt"'), (1, '0.010*"good" + 0.010*"day" + 0.008*"love" + 0.008*"free"'), (2, '0.034*"u" + 0.015*"im" + 0.009*"2" + 0.009*"ok"')]
